# Statistical sensitivity

#### Literature
See:  
S. Mertens et al  
Sensitivity of next-generation tritium beta-decay experiments for keV-scale sterile neutrinos  
JCAP02(2015)020  
https://iopscience.iop.org/article/10.1088/1475-7516/2015/02/020  

In [ ]:
import numpy as np
import plotInterface as pi; pi.init()
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from scipy.optimize import curve_fit
import scipy.stats as stats
from spectrum import *
from fastprogress.fastprogress import master_bar, progress_bar

### Calculate number of events

In [ ]:
# Statistics
measTime = 3*365*24*60*60   # 3 years
rate     = 3e8              # total rate at detector
nEvents  = rate*measTime    # expected dataset size ~10^16

# Binning
nBins       = 100
eBinEdges   = np.linspace(0,18600,nBins+1)
eBinCenters = (eBinEdges[:-1]+eBinEdges[1:])/2

print(f'Number of events: {nEvents:e}')

### The model: Tritium spectrum integrated over bins (no systematics)

In [ ]:
def model(energyBinEdges, amplitude, mSterile=0, sin2theta=0):
    
    # define function for bin integration
    spec = lambda e: diffspec_mixed(e, mSterile=mSterile, sin2theta=sin2theta)
    
    # integrate over bins
    binnedSpec = integrate_over_bins(spec,energyBinEdges)
    
    # normalize to amplitude
    binnedSpec = amplitude*binnedSpec/binnedSpec.sum()
    
    return binnedSpec

In [ ]:
# evaluate model once
yData = model(eBinEdges, nEvents)

# plot as step histogram
plt.step(eBinCenters,yData)
plt.step(eBinCenters,model(eBinEdges, nEvents, 10000, 0.3))

pi.plotty()

### Covariance matrix

* Poissonian statistical uncertainty for bin $i$: $\sigma_i=\sqrt{N_i}$
* No correlation between bins for statistical error -> $\sigma_i$ sit on diagonal of covariance matrix

In [ ]:
# calculate covariance matrix for statistical error
covStat = np.ones((nBins,nBins)) # initialize with ones to ensure inversibility
for i in range(nBins):
    covStat[i][i] = yData[i]

# for convenience also 1d error array for satstistics
yErr = np.sqrt(yData)

# invert covariance matrix
covInvStat = np.linalg.inv(covStat)

# show matrix
plt.imshow(covStat)
plt.colorbar()
plt.show()

### Calculate a chisquare value
For one particular mass and mixing amplitude 

In [ ]:
ms  = 10e3
s2t = 2e-7

# evealuate model for fixed parameters
yModel = model(eBinEdges, nEvents, mSterile=ms, sin2theta=s2t)

# define local fit model function where normalisation can be varied
def fitmodel(x,norm):
    return norm*yModel

# fit the normalisation
par, cov = curve_fit(fitmodel,eBinCenters,yData,sigma=yErr,p0=[1.0])

# evaluate fitmodel for fitted normalisation
yFit = fitmodel(eBinCenters,*par)

# calculate chisquare
yResidual = yData - yFit        
chisquareStat= yResidual@covInvStat@yResidual

# plot for demonstration
fig,ax = plt.subplots(2,1,sharex=True,height_ratios=[2, 1])

# plot spectra
ax[0].step(eBinCenters,yData, label='Data proxy (no sterile)')
ax[0].step(eBinCenters,yFit,  
           label=fr'Fit model with hypothesis $m_s={ms/1000:.0f}\,\mathrm{{keV}}, \sin^\theta={s2t:.0e}$')

# plot normalized residuals
ax[1].plot(eBinCenters,yResidual/yErr,'.',color='k')
ax[1].axhspan(-1,1,color='0.9',label='$1\sigma$ Interval')

pi.plotty(axes=ax, size=[8,6], legend=True, fontsizeLegend=10, title=[fr'$\chi^2={chisquareStat:.2f}$',None],
          ylabel=['event counts','norm.\nresiduals'], xlabel=[None,'energy(eV)'])

### Define grid over masses and mixing amplitudes

In [ ]:
# define grid over mixing angles and sterile masses
mixings = np.logspace(-5,-8,10)
masses  = np.linspace(eBinEdges[0],eBinEdges[-1],20)

plt.plot(*np.meshgrid(masses/1000,mixings), marker='x', ms=5, color="C3", ls='None')
pi.plotty(xlabel=r'$m_\mathrm{s}$ (keV)', ylabel=r'$\sin^2\theta$',log='y',ylim=[mixings[-1]/2,mixings[0]*2])

### Calculate chisquare for each grid point

In [ ]:
# invert covariance matrix
covInvStat = np.linalg.inv(covStat)

# calculate chisquare over grid
chisquareStat = np.zeros((len(mixings),len(masses)))

# set up nested progress bar
mb = master_bar(range(len(masses)),total_time=True)
mb.main_bar.comment = 'ms'
pb = progress_bar(range(len(mixings)), parent=mb)
mb.child.comment = 's2t'

for i in mb:
    for j in pb:
        ms  = masses[i]
        s2t = mixings[j]
        
        # evealuate model for fixed parameters
        yModel = model(eBinEdges, nEvents, mSterile=ms, sin2theta=s2t)
        
        # define local fit model function where normalisation can be varied
        def fitmodel(x,norm):    
            return norm*yModel
        
        # fit the normalisation
        par, cov = curve_fit(fitmodel,eBinCenters,yData,sigma=yErr,p0=[1.0])
        
        # evaluate fitmodel for fitted normalisation
        yFit = fitmodel(eBinCenters,*par)
        
        # calculate chisquare
        yResidual = yData - yFit        
        chisquareStat[j][i] = yResidual@covInvStat@yResidual

In [ ]:
fig,ax=plt.subplots(1,1)

# plot as chisquare as color mesh 
im = plt.pcolormesh(masses/1000, mixings, chisquareStat, norm=colors.SymLogNorm(2,base=10), shading='nearest')
# try: shading='nearest' actual values at the grid points
#      shading='gouraud' for interpolation

# add a colorbar
cbar = plt.colorbar(im, pad=0.03)
cbar.set_label(r'$\chi^2$',fontsize=16, rotation=0, labelpad=10)

# draw grid points
plt.plot(*np.meshgrid(masses/1000,mixings), marker='x', ms=5, color="C3", ls='None')

# show plot with labels
pi.plotty(axes=[ax], xlabel=r'$m_\mathrm{s}$ (keV)', ylabel=r'$\sin^2\theta$',log='y')

### Critical chisquare value

In [ ]:
#degrees of freedom, here: 2 (mass and mixing amplitude)
dof = 2
# confidence level 95%
CL  = 0.95

# critical chisquare value
chiSquareCrit = stats.chi2.ppf(CL, df=dof)
print(f'Critical value for {CL*100:.0f}% confidence: {chiSquareCrit}')
plt.axvline(chiSquareCrit, color='C3', label=fr'$\chi_\mathrm{{crit}}^2={chiSquareCrit:.2f}$')

# plot chisSqure distribution
x = np.linspace(0,10,1000)
y = stats.chi2.pdf(x, df=dof)
plt.plot(x,y)

# plot region
mask = x<=chiSquareCrit
plt.fill_between(x[mask], y[mask],alpha=0.5, label=f'integral = {CL*100:.0f}%')

pi.plotty(ylim=[0,None],xlim=[0,x[-1]], legend=True)

### Exclusion / Sensitivity plot

In [ ]:
fig,ax=plt.subplots(1,1)

# plot as chisquare as color mesh and a colorbar
im = plt.pcolormesh(masses/1000, mixings, chisquareStat, norm=colors.SymLogNorm(2,base=10), shading='gouraud')
cbar = plt.colorbar(im, pad=0.03)
cbar.set_label(r'$\chi^2$',fontsize=16, rotation=0, labelpad=10)
cbar.ax.plot([0, 1], [chiSquareCrit]*2, 'r',lw=3)
cbar.ax.text(1.2, chiSquareCrit*0.85, f'{chiSquareCrit:.2f}',color='r')


# plot 90% exclusion contour
ax.contour(masses/1000, mixings, chisquareStat, 
            levels=[chiSquareCrit], colors='r', linewidths=3)

# show plot with labels
pi.plotty(axes=[ax], xlabel=r'$m_\mathrm{s}$ (keV)', ylabel=r'$\sin^2\theta$',log='y')